In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import json
import os
import numpy as np
import shutil # For sorting files

In [2]:
SORTED_DATA_DIR = '../data/user2/train_sorted'


IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR_USER2 = SORTED_DATA_DIR # Use the new sorted directory

datagen_u2 = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='training'
)
validation_generator_u2 = datagen_u2.flow_from_directory(
    DATA_DIR_USER2,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # Changed to binary
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [14]:
model_v2 = Sequential([
    # Block 1
    SeparableConv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Block 2
    SeparableConv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),

    Dense(64, activation='relu'),
    Dropout(0.3),   # slightly lower dropout to reduce underfitting

    Dense(1, activation='sigmoid')
])

In [15]:
model_v2.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ separable_conv2d_3              │ (None, 126, 126, 32)   │           155 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_4              │ (None, 61, 61, 64)     │         2,400 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,164 (27.98 KB)

 Trainable params: 6,972 (27.23 KB)

 Non-trainable params: 192 (768.00 B)

In [16]:
model_v2.compile(optimizer='adam',
                  loss='binary_crossentropy', # Changed for binary
                  metrics=['accuracy'])

In [17]:
history_v2 = model_v2.fit(train_generator_u2, epochs=5, validation_data=validation_generator_u2)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 232s 363ms/step - accuracy: 0.6291 - loss: 0.6336 - val_accuracy: 0.5260 - val_loss: 0.7356
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 241s 385ms/step - accuracy: 0.6800 - loss: 0.5926 - val_accuracy: 0.6636 - val_loss: 0.6227
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 214s 343ms/step - accuracy: 0.6998 - loss: 0.5746 - val_accuracy: 0.7044 - val_loss: 0.5672
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 214s 343ms/step - accuracy: 0.7102 - loss: 0.5600 - val_accuracy: 0.7340 - val_loss: 0.5384
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 214s 343ms/step - accuracy: 0.7258 - loss: 0.5462 - val_accuracy: 0.7198 - val_loss: 0.5640


In [18]:

# --- Print metrics before saving ---
print("\n===== TRAIN METRICS =====")
if "loss" in history_v2.history:
    print(f"Final Train Loss      : {history_v2.history['loss'][-1]:.4f}")
if "accuracy" in history_v2.history:
    print(f"Final Train Accuracy  : {history_v2.history['accuracy'][-1]:.4f}")

print("\n===== VALIDATION METRICS =====")
if "val_loss" in history_v2.history:
    print(f"Final Val Loss        : {history_v2.history['val_loss'][-1]:.4f}")
if "val_accuracy" in history_v2.history:
    print(f"Final Val Accuracy    : {history_v2.history['val_accuracy'][-1]:.4f}")



===== TRAIN METRICS =====
Final Train Loss      : 0.5462
Final Train Accuracy  : 0.7258

===== VALIDATION METRICS =====
Final Val Loss        : 0.5640
Final Val Accuracy    : 0.7198


In [19]:
# --- Ensure directories exist ---
os.makedirs("../models", exist_ok=True)
os.makedirs("../results", exist_ok=True)

# --- Save model ---
model_v2.save("../models/model_v2.h5")
print("\nSaved model_v2.h5 to models/")

# --- Save ALL metrics (not only last epoch) ---
metrics_v1_data = {}

for key, values in history_v2.history.items():
    # Convert numpy.float32 → Python float
    metrics_v1_data[key] = [float(v) for v in values]

# Save final epoch separately for convenience
metrics_v1_data["final"] = {
    "train_accuracy": float(history_v2.history["accuracy"][-1]) if "accuracy" in history_v2.history else None,
    "train_loss": float(history_v2.history["loss"][-1]) if "loss" in history_v2.history else None
  }

with open("../results/metrics_v2.json", "w") as f:
    json.dump(metrics_v1_data, f, indent=4)

print("Saved metrics_v2.json to results/")


Saved model_v2.h5 to models/
Saved metrics_v2.json to results/
